# Домашнее задание 2

Это домашнее задание по материалам второго семинаров. Дедлайн по отправке - 23:55 17 февраля. 

Домашнее задание выполняется в этом же Jupyter Notebook'e и присылается мне на почту: __beznosikov.an@phystech.edu__.

Решение каждой задачи необходимо поместить после её условия.

Файл должен называться: Фамилия_Имя_Optimization_HW_2

При полном запуске Вашего решения (Kernel -> Restart & Run All) все ячейки должны выполняться без ошибок. 

## Задача 1

Рассмотрим проекцию на единичном $\ell_1$-шар:
\begin{equation*}
    \text{proj}_{C}(x) = \arg\min_{y \in C} \|x-y\|^2,
\end{equation*}
где $C = \{x ~|~ \|x\|_1 \leq 1 \}$.

__(а)__ Найдите выражение для такой проекции в явном виде.

Алгоритм проекции:
0. Считаем l1 норму вектора x, если меньше или равно. то выдаем x, иначе переходим к шагу.
1. Смотрим на координаты вектора x, из них понимаем в какой квадрат d-мерном пространстве мы попали, из этого понимаем уравнение ближайшей гиперплоскости.
2. Из уравнения гиперплоскости мы знаем нормаль к ней, можем найти расстояние от точки x до нашей гиперплоскости.
3. Нашли расстояние $h = \frac{|(x-ro, n)|}{|n|}$, где (r0 = ($(-1)^l$, 0, ..., 0), а $l = sign(x_1)$), тогда можем найти в какой точке перпендикуляр из точки x пересекает нашу гиперплоскость.
4. Эта точка будет задаваться выражением: $y = x - h \cdot n$.
5. Вычисляем l1 норму вектора y, если он больше единицы, то перебираем оставшиеся d вершин нашего многогранника (гиперплоскости) и выбираем ту, до которой минимальное расстояние.


__(б)__ Сравните:
\begin{equation*}
    \text{proj}_C (x) \quad \text{и} \quad \text{prox}_{\lambda \mathbb{I}_C} (x),
\end{equation*}
где $\lambda > 0$, а $\mathbb{I}_C$ - индикаторная функция множества $C$:
$$
\mathbb{I}_C = 
\begin{cases}
0 ,& x \in C \\
+\infty, &x \notin C.
\end{cases}
$$
Являются ли данные операторы эквивалентными для любых $\lambda > 0$ и выпуклых $C$?

In [76]:
#ответ


## Задача 2

Рассмотрим задачу минимизации эмпирического риска:
\begin{equation}
\min_{w \in \mathbb{R}^d} \frac{1}{n} \sum\limits_{i=1}^n l (g(w, x_i), y_i),
\end{equation}
где $l$ - функция потерь, $g$ - модель, $w$ - параметры модели, $\{x_i, y_i\}_{i=1}^n$ - выборка данных из векторов признаков $x_i$ и меток $y_i$.

Далее будем рассматривать линейной модель $g(w, x) = w^T x$ и квадратичную функцию потерь $l(z, y) = (z-y)^2$.

__(а)__ Запишите задачу минимизации эмпирического риска с линейной моделью  и квадратичной функцией потерь в векторно-матричном виде, используя $X$ - матрицу из векторов $\{x_i\}$ и вектор $y$ - из $\{y_i\}$.

\begin{equation}
\min_{w \in \mathbb{R}^d} ||w^TX - y||_2, X - \text{ матрица из векторов}, y - \text{вектор из }\{y_i\}.
\end{equation}

__(б)__ К заданию приложен датасет _mushrooms_. С помощью следующего кода сформируйте матрицу $X$ и вектор $y$:

In [77]:
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
import scipy.linalg as la
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

In [78]:
dataset = "mushrooms.txt" 
#файл должен лежать в той же деректории, что и notebook

In [79]:
data = load_svmlight_file(dataset)
X, y = data[0].toarray(), data[1]
n, d = X.shape
print(X.shape, y.shape)

(8124, 112) (8124,)


Разделите (лучше случайно) данные (а значит $X$ и $y$) на две части: обучающую и тестовую (примерно в отношении $4$ к $1$). Опишите, как это было сделано.

In [80]:
#ответ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [81]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(6499, 112) (6499,)
(1625, 112) (1625,)


__(в)__ Для обучающей части $X_{train}$, $y_{train}$ оцените константу $L$ задачи обучения/оптимизации. Будет ли задача выпуклой? Сильно выпуклой?

In [82]:
#ответ
n, k = X_train.shape
hessian = 2/n * X_train.T @ X_train
wb, vb = np.linalg.eigh(hessian)
print("Minimal eigenvalue: ", wb[0])
print("Maximal eigenvalue: ", wb[-1])

Minimal eigenvalue:  -3.168735668875253e-14
Maximal eigenvalue:  20.677110902977507


Понятно, что задача будет выпуклой, т.к. у нас квадрат евклидовой нормы это выпуклая функция, а внутри нормы стоит аффинная функция, поэтому их композиция будет выпуклой функцией.

$L = 20.67$

Формально функция потерь является сильно выпуклой функцией, но как мы поняли в прошлом задании, $\mu = 10^{-5}$ это будет плохо обусловленная матрица, а с $\mu = 10^{-12}$ будет совсем плохо обусловленная матрица, поэтому из теории считаем, что да, но на практике нет.

__(г)__ Используя материалы прошлого ДЗ (по желанию), решите задачу оптимизации/обучите модель машинного обучения с помощью градиентного спуска. Изобразите график сходимости. Какой критерий сходимости будет использовать? Сделайте вывод. Опишите проблемы, с которыми пришлось столкнуться.

In [88]:
#@title Класс линейной регрессии
class MyLinearRegression:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
    
    def __function(self, X, y, w, n):
        return 1/n * la.norm(w @ X.T - y, 2) ** 2

    def __grad_function(self, X, y, w, n):
        """
        Функция под
        """
        return 1/n * 2 * X.T @ (w @ X.T - y).T 
        
    def fit(self, X, y):
        """
        Функция подбора параметров линейной модели для квадратичной функции потерь.

        Input: 
        - X     : матрица объектов
        - y     : вектор ответов

        Returns: none.
        """

        n, k = X.shape
        wo = np.array([])
        X_train = X

        # Добавляем ещё столбец для константы
        if self.fit_intercept:
            X_train = np.hstack((X, np.ones((n, 1))))
            w0 = np.random.rand(k+1)
        else:
            w0 = np.random.rand(k)
        

        iter = int(10 ** 7)
        
        hessian = 2 / n * X_train.T @ X_train
        wb, vb = np.linalg.eigh(hessian)
        
        lr_func = lambda X, y, w, n: 1/wb[-1]
        
        error_criterion = lambda X, y, w, n: np.linalg.norm(self.__grad_function(X, y, w, n), 2)
        
        for iter_k in range(1000, iter, 1000):
            self._w = self.__gradient_descent(self.__function, self.__grad_function,
                                            w0, lr_func,  iter, error_criterion, 
                                            X_train, y)
            
            error = error_criterion(X, y, self._w, n)
            if (error < eps): break

        
        return self
    
    def predict(self, X):
        """
        Функция предсказания по признакам, уже натренированной модели.
        Input: 
        - X : объекты, по которым будем предксазывать

        Returns: предсказания на основе весов, ранее обученной линейной модели.
        """
        n, k = X.shape
        X_train = X
        if self.fit_intercept:
            X_train =np.hstack((X, np.ones((n, 1))))

        y_pred = self._w @ X_train.T

        return y_pred

    def get_weights(self):
        """
        Функция получения весов нашей линейной модели.

        Input: None.
        Returns: Параметры модели.
        """
        return self._w

    def __gradient_descent(self, f, grad_f, x0, 
                         lr, iter, error_criterion, 
                         X, y):
        """
        Это градиентный спуск.
        Он получает на вход целевую функцию, функцию градиента целевой функции, 
        начальную точку, функцию learning rate, количество итераций и 
        функцию подсчета ошибки. И применяетметод градиентного спуска.

        Inputs:
        - f                 : целевая функция, минимум которой мы хотим найти.
        - grad_f            : функция градиента целевой функции.
        - x0                : начальная точка.
        - lr                : функция learning rate.
        - iter              : количество итераций.
        - error_criterion   : функция подсчета ошибки
        - X_train           : множество объектов (матрица фичей)
        - y                 : вектор ответов

        Returns:
        Наилучшую минимальную точку, которую удалось найти.
        """
    
        w = x0
        eps = 5*10e-5
        n, k = X.shape 
        for k in range(iter):
            prev_w = w
            w = w - lr(X, y, w, n) * grad_f(X, y, w, n)
            
            error = error_criterion(X, y, w, n)

            if (error < eps):
                return w
        return w

__(д)__ Как использовать итоговую модель для предсказания? Ответив на вопрос, сделайте предсказания на тестовой выборке. Сравните с реальными метками. Количество правильно угаданных меток есть точность/accuracy вашей модели. Запустите еще раз процесс обучения из пункта г), постройте как меняется точность модели от номера итерации градиентного спуска (измеряйте не каждую итерацию, а, например, каждую 100 или 1000). 

In [90]:
import time
start = time.time()
linear_model = MyLinearRegression(True)
linear_model.fit(X_train, y_train)
end = round(time.time() - start, 5)

KeyboardInterrupt: 

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred_sklearn = model.predict(X_test)
y_pred_my = linear_model.predict(X_test)
y_pred_sklearn = np.round(y_pred_sklearn)
y_pred_my = np.round(y_pred_my)


r2_score_sklearn = r2_score(y_test, y_pred_sklearn)
r2_score_my = r2_score(y_test, y_pred_my)

mean_squared_error_sklearn = mean_squared_error(y_test, y_pred_sklearn)
mean_squared_error_my = mean_squared_error(y_test, y_pred_my)

accuracy_score_sklearn = accuracy_score(y_test, y_pred_sklearn)
accuracy_score_my = accuracy_score(y_test, y_pred_my)

print("               : sklearn | my regresion")
print("R2 score       : %0.5f | %0.5f  " % (r2_score_sklearn, r2_score_my))
print("mean_scored    : %0.5f | %0.5f  " % (mean_squared_error_sklearn, mean_squared_error_my))
print("accuracy_score : %0.5f | %0.5f  " % (accuracy_score_sklearn, accuracy_score_my))

__(e)__ Теперь рассмотрим эту же задачу обучения на $\ell_1$-шаре с радиусом $\lambda$. Повторите пункты г) и д) только теперь градиентный спуск должен быть модифицирован с помощью оператора проекции. Пробуйте различные $\lambda > 0$ и найдите лучшее (опишите почему именно такое лучшее). Удалось ли улучшить процесс обучения/точность модели на тестовой выборке? 

In [97]:
class MyLinearRegressionProj:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
    
    def __function(self, X, y, w, n):
        return 1/n * la.norm(w @ X.T - y, 2) ** 2

    def __grad_function(self, X, y, w, n):
        """
        Функция градиента функции.

        Input:
        - X         : матрица фичей
        - y         : вектор ответов
        - w         : вектор весов модели
        - n         : количество экспериментов в матрице
        """
        return 1/n * 2 * X.T @ (w @ X.T - y).T 
        

    def __lr_func(self, k, w, y, X, f, grad_f, n):        
        return 1 / self.Lr

    def fit(self, X, y):
        """
        Функция подбора параметров линейной модели для квадратичной функции потерь.

        Input: 
        - X     : матрица объектов
        - y     : вектор ответов

        Returns: none.
        """

        n, k = X.shape
        wo = np.array([])
        X_train = X

        if self.fit_intercept:
            X_train = np.hstack((X, np.ones((n, 1))))
            w0 = np.random.rand(k+1)
        else:
            w0 = np.random.rand(k)
        
        iter = int(10 ** 7)
        hessian = 2 / n * X_train.T @ X_train
        wb, vb = np.linalg.eigh(hessian)
        
        self.Lr = wb[-1] + wb[0]
        
        #error_criterion = lambda X, y, w, n : np.linalg.norm(self.__grad_function(X, y, w, n), 2) ** 2
        error_criterion = lambda X, y, w, n, w_prev : np.linalg.norm(w - w_prev)
        self._w = self.__gradient_descent(self.__function, self.__grad_function,
                                        w0, self.__lr_func,  iter, 
                                        error_criterion, X_train, y)

        
        return self
    
    def predict(self, X):
        """
        Функция предсказания по признакам, уже натренированной модели.
        Input: 
        - X : объекты, по которым будем предксазывать

        Returns: предсказания на основе весов, ранее обученной линейной модели.
        """
        n, k = X.shape
        X_train = X
        if self.fit_intercept:
            X_train =np.hstack((X, np.ones((n, 1))))

        y_pred = self._w @ X_train.T

        return y_pred

    def get_weights(self):
        """
        Функция получения весов нашей линейной модели.

        Input: None.
        Returns: Параметры модели.
        """

        return self._w

    def proj(self, vector_x):
        if (la.norm(vector_x, 1) <= 1):
            return vector_x
        
        vector_n = np.sign(vector_x)
        vector_n = vector_n / la.norm(vector_n, 2)

        #print("Vector normal:", vector_n)

        vector_r0 = np.zeros(len(vector_x))
        
        for i in range(len(vector_x)):
            if vector_x[i] != 0:
                vector_r0[i] = np.sign(vector_x[i])
                break
        #print("Vector R0:", vector_r0)

        h = np.abs( (vector_x - vector_r0) @ vector_n) / la.norm(vector_n, 2)
        #print("Distance to plain: ", h)
        
        vector_y = vector_x - h * vector_n
        eps = 10e-6

        #print("Vector_y (1): ", vector_y, " norm_1 = ", la.norm(vector_y, 1))
        if (np.abs(la.norm(vector_y, 1) - 1) < eps):
            return vector_y
        else:
            vector_y = vector_x + h * vector_n
            #print("Vector_y (2): ", vector_y, " norm_1 = ", la.norm(vector_y, 1))
            if (np.abs(la.norm(vector_y, 1) - 1) < eps):
                return vector_y
        
        vector_vert = np.zeros(len(vector_x))
        vector_vert[0] = np.sign(vector_x[0])
        
        min_dist = la.norm(vector_x - vector_vert, 2)
        min_vert = 0

        for i in range(1, len(vector_x) - 1, 1):
            vector_vert[i - 1] = 0
            vector_vert[i] = np.sign(vector_x[i])
            dist_i = la.norm(vector_x - vector_vert, 2)
            if (dist_i < min_dist):
                min_vert = i
                min_dist = dist_i
        
        vector_proj = np.zeros(len(vector_x))
        vector_proj[min_vert] = np.sign(vector_x[min_vert])
        
        return vector_proj


    def __gradient_descent(self, f, grad_f, x0, 
                         lr, iter, error_criterion, 
                         X, y):
        """
        Это градиентный спуск.
        Он получает на вход целевую функцию, функцию градиента целевой функции, 
        начальную точку, функцию learning rate, количество итераций и 
        функцию подсчета ошибки. И применяетметод градиентного спуска.

        Inputs:
        - f                 : целевая функция, минимум которой мы хотим найти.
        - grad_f            : функция градиента целевой функции.
        - x0                : начальная точка.
        - lr                : функция learning rate.
        - iter              : количество итераций.
        - error_criterion   : функция подсчета ошибки
        - X_train           : множество объектов (матрица фичей)
        - y                 : вектор ответов

        Returns:
        Наилучшую минимальную точку, которую удалось найти.
        """
    
        w = x0
        eps = 10e-7
        n, k = X.shape 
        for k in range(iter):
            prev_w = w
            w = w - lr(k, w, y, X, f, grad_f, n) * self.proj(grad_f(X, y, w, n))
            
            error = error_criterion(X, y, w, n, prev_w)

            if (error < eps):
                return w
            if (k % 1000 == 0):
                print("%d - %f" % (k, error))
        print("Final number of operations: %d" % (k))
        return w

In [99]:
import time
start = time.time()
linear_model = MyLinearRegressionProj(True)
linear_model.fit(X_train, y_train)
end = round(time.time() - start, 5)

0 - 0.044123
1000 - 0.001184
2000 - 0.000478
3000 - 0.000268
4000 - 0.000171
5000 - 0.000119
6000 - 0.000089
7000 - 0.000071
8000 - 0.000060
9000 - 0.000052
10000 - 0.000045
11000 - 0.000040
12000 - 0.000036
13000 - 0.000033
14000 - 0.000030
15000 - 0.000027
16000 - 0.000025
17000 - 0.000023
18000 - 0.000021
19000 - 0.000020
20000 - 0.000018
21000 - 0.000017
22000 - 0.000016
23000 - 0.000015
24000 - 0.000014
25000 - 0.000014
26000 - 0.000013
27000 - 0.000012
28000 - 0.000011
29000 - 0.000011
30000 - 0.000010
31000 - 0.000010
32000 - 0.000009
33000 - 0.000009
34000 - 0.000009
35000 - 0.000008
36000 - 0.000008
37000 - 0.000008
38000 - 0.000007
39000 - 0.000007
40000 - 0.000007
41000 - 0.000007
42000 - 0.000006
43000 - 0.000006
44000 - 0.000006
45000 - 0.000006
46000 - 0.000006
47000 - 0.000005
48000 - 0.000005
49000 - 0.000005
50000 - 0.000005
51000 - 0.000005
52000 - 0.000005
53000 - 0.000005
54000 - 0.000004
55000 - 0.000004
56000 - 0.000004
57000 - 0.000004
58000 - 0.000004
59000 - 0.

In [95]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [96]:
y_pred_sklearn = model.predict(X_test)
y_pred_my = linear_model.predict(X_test)
y_pred_sklearn = np.round(y_pred_sklearn)
y_pred_my = np.round(y_pred_my)


r2_score_sklearn = r2_score(y_test, y_pred_sklearn)
r2_score_my = r2_score(y_test, y_pred_my)

mean_squared_error_sklearn = mean_squared_error(y_test, y_pred_sklearn)
mean_squared_error_my = mean_squared_error(y_test, y_pred_my)

accuracy_score_sklearn = accuracy_score(y_test, y_pred_sklearn)
accuracy_score_my = accuracy_score(y_test, y_pred_my)

print("               : sklearn | my regresion")
print("R2 score       : %0.5f | %0.5f  " % (r2_score_sklearn, r2_score_my))
print("mean_scored    : %0.5f | %0.5f  " % (mean_squared_error_sklearn, mean_squared_error_my))
print("accuracy_score : %0.5f | %0.5f  " % (accuracy_score_sklearn, accuracy_score_my))

               : sklearn | my regresion
R2 score       : 1.00000 | 0.99753  
mean_scored    : 0.00000 | 0.00062  
accuracy_score : 1.00000 | 0.99938  


__Бонусные пункт__

__(ж)__ Снова рассмотрим задачу обучения без ограничений, но добавим к ней регуляризатор 1) $\lambda \| x \|_1$ c $\lambda  > 0$, 2) $\lambda \| x \|_2$ c $\lambda  > 0$. Сможем ли для новой задачи с регуляризатором использовать честный градиентный спуск без модификаций? Почему? Предложите свои идеи, как модифицировать градиентный спуск для новой задачи. Проведите для него эксперименты из пунктов г) и д).  Пробуйте различные $\lambda > 0$ и найдите лучшее (опишите почему именно такое лучшее). Сравните результаты с пунктом e).

In [ ]:
#ответ

__(з)__ Воспользуемся для нашей задачи методом [PCA](https://ru.wikipedia.org/wiki/Метод_главных_компонент). Прочитайте про него подробнее и опишите основную идею.

In [ ]:
#ответ

Следующий код запускает PCA для нашего датасета. Убедитесь, что понимаете происходящие.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
pca.fit(X_train)  #эта строчка будет работать только если есть матрица X_train
print (pca.explained_variance_ratio_.cumsum())

Какое значение n_components разумно поставить? Для новой матрицы X_train повторите пункт ж). Измерьте время работы процесса обучения. Постройте график точности решения на тестовой выборке от времени работы для честного X_train и полученного с помощью PCA.